In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

C:\Users\shrad\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [75]:
acq_data = ['Id', 'chan','Name', 'IntRate', 'UnpaidPrinciple', 'LoanDur', 'date_1', 'Payment_one', 'LTV', 'CLTV',
           'Borrow', 'DTI', 'credit_score', 'homeloan','loanreason', 'prop_type', 'No_units', 'status', 'state','address',
           'Mortgage','type','creditscore1', 'Mortgagetype','Mort'];


perf_data = ['Id','month','col3','curintrate','col5','Age','maturity_date','remaining_time','col9','col10','col11','flag','col13',
            'date','lastpaymentdate','closureDate','col17','col18','col19','col20','col21','col22','col23','col24','col25',
            'col26','col27','col28','col29']



In [76]:
processed_acq = pd.read_csv('2000Q1'+'/Acquisition_2000Q1.txt', sep = '|',names = acq_data,index_col=False)
processed_per = pd.read_csv('2000Q1'+'/Performance_2000Q1.txt',sep = '|', names = perf_data, usecols=[0, 15], index_col=False)


In [77]:
print processed_per

                   Id closureDate
0        100007365142         NaN
1        100007365142         NaN
2        100007365142         NaN
3        100007365142         NaN
4        100007365142         NaN
5        100007365142         NaN
6        100007365142         NaN
7        100007365142         NaN
8        100007365142         NaN
9        100007365142         NaN
10       100007365142         NaN
11       100007365142         NaN
12       100007365142         NaN
13       100007365142         NaN
14       100007365142         NaN
15       100007365142         NaN
16       100007365142         NaN
17       100007365142         NaN
18       100007365142         NaN
19       100007365142         NaN
20       100007365142         NaN
21       100007365142         NaN
22       100007365142         NaN
23       100007365142         NaN
24       100007365142         NaN
25       100007365142         NaN
26       100007365142         NaN
27       100007365142         NaN
28       10000

In [78]:
processed_per.drop_duplicates(subset='Id', keep='last', inplace=True)
new_data = pd.merge(processed_acq, processed_per, on='Id', how='inner')

#new_data.rename(index=str, columns={"closureDate": 'Default'}, inplace=True)


In [79]:
print new_data['closureDate']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
5                NaN
6                NaN
7                NaN
8                NaN
9                NaN
10               NaN
11               NaN
12               NaN
13               NaN
14               NaN
15               NaN
16               NaN
17               NaN
18               NaN
19               NaN
20               NaN
21               NaN
22               NaN
23               NaN
24               NaN
25               NaN
26               NaN
27               NaN
28               NaN
29               NaN
             ...    
246833           NaN
246834           NaN
246835           NaN
246836           NaN
246837           NaN
246838           NaN
246839           NaN
246840           NaN
246841           NaN
246842           NaN
246843           NaN
246844           NaN
246845           NaN
246846           NaN
246847           NaN
246848           NaN
246849    03/

In [58]:
print new_data['closureDate']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
5                NaN
6                NaN
7                NaN
8                NaN
9                NaN
10               NaN
11               NaN
12               NaN
13               NaN
14               NaN
15               NaN
16               NaN
17               NaN
18               NaN
19               NaN
20               NaN
21               NaN
22               NaN
23               NaN
24               NaN
25               NaN
26               NaN
27               NaN
28               NaN
29               NaN
             ...    
246833           NaN
246834           NaN
246835           NaN
246836           NaN
246837           NaN
246838           NaN
246839           NaN
246840           NaN
246841           NaN
246842           NaN
246843           NaN
246844           NaN
246845           NaN
246846           NaN
246847           NaN
246848           NaN
246849    03/

In [80]:
new_data['closureDate'].fillna(0,inplace=True)
length =  len(new_data['closureDate'])
for i in range(0,length):
    if new_data['closureDate'][i] != 0:
        new_data['closureDate'][i] = 1

new_data['closureDate'] = new_data['closureDate'].astype(int)

C:\Users\shrad\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [81]:
print new_data

                  Id chan                                       Name  IntRate  \
0       100007365142    R                    JPMORGAN CHASE BANK, NA    8.000   
1       100007386460    B                    JPMORGAN CHASE BANK, NA    7.875   
2       100011322040    C                               AMTRUST BANK    7.750   
3       100015192562    R                                      OTHER    8.500   
4       100015874399    C                         CITIMORTGAGE, INC.    8.750   
5       100017922445    C                               AMTRUST BANK    8.250   
6       100019048933    C                                      OTHER    7.500   
7       100020205696    R  FIRST TENNESSEE BANK NATIONAL ASSOCIATION    7.625   
8       100021703104    C                    JPMORGAN CHASE BANK, NA    8.000   
9       100023274028    R  FIRST TENNESSEE BANK NATIONAL ASSOCIATION    8.000   
10      100026859568    C  FIRST TENNESSEE BANK NATIONAL ASSOCIATION    8.375   
11      100028993931    R   

In [82]:
new_data['dateMonth'] = new_data['date_1'].apply(lambda a: a.split('/')[0].strip()).astype(int)
new_data['dateYear'] = new_data['date_1'].apply(lambda a: a.split('/')[1].strip()).astype(int)

new_data['paymentMonth'] =  new_data['Payment_one'].apply(lambda a: a.split('/')[0].strip()).astype(int)
new_data['paymentYear'] =  new_data['Payment_one'].apply(lambda a: a.split('/')[1].strip()).astype(int)


new_data.drop(['date_1','Payment_one'], axis=1, inplace =True)


In [83]:
print new_data

                  Id chan                                       Name  IntRate  \
0       100007365142    R                    JPMORGAN CHASE BANK, NA    8.000   
1       100007386460    B                    JPMORGAN CHASE BANK, NA    7.875   
2       100011322040    C                               AMTRUST BANK    7.750   
3       100015192562    R                                      OTHER    8.500   
4       100015874399    C                         CITIMORTGAGE, INC.    8.750   
5       100017922445    C                               AMTRUST BANK    8.250   
6       100019048933    C                                      OTHER    7.500   
7       100020205696    R  FIRST TENNESSEE BANK NATIONAL ASSOCIATION    7.625   
8       100021703104    C                    JPMORGAN CHASE BANK, NA    8.000   
9       100023274028    R  FIRST TENNESSEE BANK NATIONAL ASSOCIATION    8.000   
10      100026859568    C  FIRST TENNESSEE BANK NATIONAL ASSOCIATION    8.375   
11      100028993931    R   

In [84]:
new_data.drop(['Mortgage','Mortgagetype','creditscore1','type','Id'], axis=1, inplace=True)